In [3]:
import os
import sys
sys.path.append("../../")
import argparse

import pandas as pandas
import numpy as np

import torch
import matplotlib.pyplot as plt

from utils import load_config, load_model, get_representation_hook

In [4]:
def get_representation_hook(name):
    def hook(model, input, output):
        representations[name] = output.detach()

    return hook

In [5]:
agg_method = 'FedProx'
proj_name = 'dashing-gorge-7'
ckptPATH = f'/NFS/Users/moonsh/AdaptFL/ckpt/{agg_method}/' # 'Z://Users/moonsh/AdaptFL/ckpt'

config = load_config(ckptPATH, proj_name)
config['batch_size'] = 64
config['num_workers'] = 4
config['nowandb'] = True

config = argparse.Namespace(**config)

In [6]:
main_name = proj_name.split('-')[1]
device = torch.device("cuda:0")

model_name = 'dashing-gorge-7_best_round_048.pth'
glob_model, loc_model_dict = load_model(model_name, ckptPATH, config, device)

In [ ]:
for k, v in glob_model.named_parameters():
    print(k, v.shape)

In [ ]:
representations = {}
glob_model.layer1.register_forward_hook(get_representation_hook('layer1'))
glob_model.layer2.register_forward_hook(get_representation_hook('layer2'))
glob_model.layer3.register_forward_hook(get_representation_hook('layer3'))
glob_model.layer4.register_forward_hook(get_representation_hook('layer4'))
glob_model.fc.register_forward_hook(get_representation_hook('fc'))

In [32]:
data = torch.randn(1, 1, 96, 128, 96).to(device)
with torch.no_grad():
    output = glob_model(data)

In [ ]:
for layer, rep in representations.items():
    print(f"Representation from {layer}: {rep.shape}")